# Diabetes Readmission modelling using Decission Trees

In [0]:
# Data processing
from pyspark.sql.functions import log, col, exp

# Modeling
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [0]:
diabetes_readmit = spark.read.table("diab_readmit_csv")
write_path = 'dbfs:/tmp/reproducible_ml_uofl/diab_readmit_csv.delta'
diabetes_readmit.write.format('delta').mode('overwrite').save(write_path)

In [0]:
diabetes_readmit = spark.read.format('delta').load(write_path)

#Show basic summary stats
display(diabetes_readmit.summary())

summary,patient_nbr,time_in_hospital,num_procedures,num_lab_procedures,num_medications,number_outpatient,number_inpatient,number_emergency,number_diagnoses,gender_cd,DiabetesMedication,readmit_flag,race_cd
count,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766
mean,5.4330400694947235E7,4.395986871843248,1.339730361810428,43.09564098028811,16.021844230882614,0.36935715268360747,0.635565906098304,0.19783621248747127,7.422606764538254,0.5375862272271682,0.7700312481575379,0.11159915885462728,null
stddev,3.869635934653421E7,2.985107767471267,1.705806979121172,19.674362249142096,8.127566209167309,1.2672650965326817,1.26286329009732,0.9304722684224632,1.9336001449974247,0.49858772375671534,0.420814525814695,0.3148741984505526,null
min,135,1,0,1,1,0,0,0,1,0,0,0,AfrAmr
25%,23412645,2,0,31,10,0,0,0,6,0,1,0,null
50%,45500490,4,1,44,15,0,0,0,8,1,1,0,null
75%,87532902,6,2,57,20,0,1,0,9,1,1,0,null
max,189502619,14,6,132,81,42,21,76,16,1,1,1,White


In [0]:
# Train test split
trainDF, testDF = diabetes_readmit.randomSplit([.65, .35], seed=42)
# Print the number of records
print(f'There are {trainDF.cache().count()} records in the training dataset.')
print(f'There are {testDF.cache().count()} records in the testing dataset.')

There are 66284 records in the training dataset.
There are 35482 records in the testing dataset.

In [0]:
#One hot encoding of string feature Region
trainDF.groupBy('race_cd').count().show()

+--------+-----+
 race_cd|count|
+--------+-----+
 Unk| 2472|
 AfrAmr|12558|
 White|49512|
Hispanic| 1326|
 Asian| 416|
+--------+-----+

##Now we need to modify the categorical variable race_cd into one-hot-encoded version
 For this we will use a pipeline to do this in one step

In [0]:
#You can also create a pipeline and do everything together in one easy fit and transform step
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
 
categoricalColumns = ["race_cd"]
stages = [] # stages in Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    
# Use OneHotEncoder to convert categorical variables into binary SparseVectors
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])

# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=[stringIndexer, encoder])
 
# Define the pipeline model.
transform_mdl = pipeline.fit(trainDF)
trainDF21=transform_mdl.transform(trainDF)
trainDF21.show()

+-----------+----------------+--------------+------------------+---------------+-----------------+----------------+----------------+----------------+---------+------------------+------------+-------+------------+---------------+
patient_nbr|time_in_hospital|num_procedures|num_lab_procedures|num_medications|number_outpatient|number_inpatient|number_emergency|number_diagnoses|gender_cd|DiabetesMedication|readmit_flag|race_cd|race_cdIndex|race_cdclassVec|
+-----------+----------------+--------------+------------------+---------------+-----------------+----------------+----------------+----------------+---------+------------------+------------+-------+------------+---------------+
 135| 3| 1| 31| 14| 0| 1| 0| 5| 1| 1| 0| White| 0.0| (4,[0],[1.0])|
 135| 8| 6| 77| 33| 0| 0| 0| 8| 1| 1| 1| White| 0.0| (4,[0],[1.0])|
 729| 4| 2| 68| 23| 0| 0| 0| 9| 1| 1| 0| White| 0.0| (4,[0],[1.0])|
 927| 5| 0| 49| 5| 0| 0| 0| 3| 1| 1| 0| AfrAmr| 1.0| (4,[1],[1.0])|
 1152| 6| 4| 45| 15| 0| 2| 0| 6| 1| 1| 0| AfrAmr| 1.0| (4,[1],[1.0])|
 1152| 12| 1| 37| 18| 0| 2| 0| 5| 1| 1| 0| AfrAmr| 1.0| (4,[1],[1.0])|
 1314| 1| 5| 47| 12| 0| 1| 0| 7| 1| 1| 1| White| 0.0| (4,[0],[1.0])|
 2025| 12| 2| 47| 18| 0| 0| 0| 9| 1| 1| 0| White| 0.0| (4,[0],[1.0])|
 3069| 8| 6| 57| 31| 0| 0| 0| 9| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 4302| 1| 1| 31| 9| 0| 0| 0| 7| 1| 0| 0| White| 0.0| (4,[0],[1.0])|
 5220| 2| 0| 15| 14| 0| 0| 0| 9| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 5220| 11| 4| 65| 19| 0| 2| 0| 9| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 6174| 7| 0| 77| 12| 0| 0| 0| 9| 1| 1| 0| White| 0.0| (4,[0],[1.0])|
 6228| 4| 4| 47| 16| 0| 0| 0| 7| 1| 0| 0| White| 0.0| (4,[0],[1.0])|
 6309| 1| 5| 35| 13| 0| 0| 0| 8| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 6939| 6| 2| 65| 19| 0| 0| 0| 7| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 8154| 1| 0| 58| 5| 0| 0| 0| 8| 1| 0| 0| White| 0.0| (4,[0],[1.0])|
 8748| 2| 0| 64| 10| 0| 0| 0| 9| 0| 1| 0| AfrAmr| 1.0| (4,[1],[1.0])|
 10377| 1| 2| 33| 12| 0| 0| 0| 5| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 10422| 4| 2| 54| 3| 0| 0| 0| 8| 1| 0| 0| AfrAmr| 1.0| (4,[1],[1.0])|
+-----------+----------------+--------------+------------------+---------------+-----------------+----------------+----------------+----------------+---------+------------------+------------+-------+------------+---------------+
only showing top 20 rows

In [0]:
trainDF21.printSchema()

root
-- patient_nbr: integer (nullable = true)
-- time_in_hospital: integer (nullable = true)
-- num_procedures: integer (nullable = true)
-- num_lab_procedures: integer (nullable = true)
-- num_medications: integer (nullable = true)
-- number_outpatient: integer (nullable = true)
-- number_inpatient: integer (nullable = true)
-- number_emergency: integer (nullable = true)
-- number_diagnoses: integer (nullable = true)
-- gender_cd: integer (nullable = true)
-- DiabetesMedication: integer (nullable = true)
-- readmit_flag: integer (nullable = true)
-- race_cd: string (nullable = true)
-- race_cdIndex: double (nullable = false)
-- race_cdclassVec: vector (nullable = true)

In [0]:
# Linear regression expect a vector input
vecAssembler = VectorAssembler(inputCols=['time_in_hospital','num_procedures','num_medications', 'number_inpatient','number_emergency','number_diagnoses','DiabetesMedication'], outputCol="features")
vecTrainDF = vecAssembler.transform(trainDF21)

In [0]:
testDF21=transform_mdl.transform(testDF) #do the data transformation using saved parameters from training
vecTestDF = vecAssembler.transform(testDF21) #do the feature transformation using vector assembler

In [0]:
# Create Decision tree calssifier
dt = DecisionTreeClassifier(featuresCol="features", labelCol="readmit_flag", maxDepth = 20)
dtModel = dt.fit(vecTrainDF)
predict_train = dtModel.transform(vecTrainDF)
predict_train.select('readmit_flag', 'rawPrediction', 'prediction', 'probability').show(10)

+------------+-------------+----------+--------------------+
readmit_flag|rawPrediction|prediction| probability|
+------------+-------------+----------+--------------------+
 0| [9.0,0.0]| 0.0| [1.0,0.0]|
 1| [375.0,35.0]| 0.0|[0.91463414634146...|
 0| [262.0,27.0]| 0.0|[0.90657439446366...|
 0| [81.0,1.0]| 0.0|[0.98780487804878...|
 0| [30.0,0.0]| 0.0| [1.0,0.0]|
 0| [22.0,0.0]| 0.0| [1.0,0.0]|
 1| [0.0,1.0]| 1.0| [0.0,1.0]|
 0| [58.0,2.0]| 0.0|[0.96666666666666...|
 0| [375.0,35.0]| 0.0|[0.91463414634146...|
 0| [356.0,10.0]| 0.0|[0.97267759562841...|
+------------+-------------+----------+--------------------+
only showing top 10 rows

In [0]:
# Make predictions on testing dataset
predict_test = dtModel.transform(vecTestDF) #make predictions using the trained model
predict_test.groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 0.0|33899|
 1.0| 1583|
+----------+-----+

In [0]:
eval = BinaryClassificationEvaluator(rawPredictionCol = "prediction", labelCol = "readmit_flag")
auc_train = eval.evaluate(predict_train)
print(auc_train)

auc_test = eval.evaluate(predict_test)
print(auc_test)

0.6480782562082884
0.5144661885938187